# **Loading the PreProcessed Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load the dataset
file_path = 'north_Final.csv'
df = pd.read_csv(file_path)
print("Head of the dataset:")
print(df.head())
print(df.shape[0])

Head of the dataset:
       hour  Amount of precipitation in millimeters (last hour)  \
0  1.372398                                          -0.149752    
1  1.516861                                          -0.149752    
2  1.661324                                          -0.149752    
3 -1.661325                                          -0.149752    
4 -1.516862                                          -0.149752    

   Atmospheric pressure at station level (mb)  \
0                                    0.561579   
1                                    0.573018   
2                                    0.630215   
3                                    0.675972   
4                                    0.727449   

   Maximum air pressure for the last hour in hPa to tenths  \
0                                           0.553969         
1                                           0.553969         
2                                           0.611175         
3                                

# **4.Random Forests Regressor Model**

### **to utilize GPU**

In [4]:
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/pip-install.py

!pip install \
    --extra-index-url=https://pypi.nvidia.com \
    cudf-cu11==24.2.* dask-cudf-cu11==24.2.* cuml-cu11==24.2.* \
    cugraph-cu11==24.2.* cuspatial-cu11==24.2.* cuproj-cu11==24.2.* \
    cuxfilter-cu11==24.2.* cucim-cu11==24.2.* pylibraft-cu11==24.2.* \
    raft-dask-cu11==24.2.*




Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.9/460.9 MB 1.9 MB/s eta 0:00:0000:0100:07
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.5 MB/s eta 0:00:0000:0100:16
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 GB 1.4 MB/s eta 0:00:0000:0100:18
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 MB 1.9 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.2/948.2 kB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.8/687.8 MB 1.6 MB/s eta 0:00:0000:0100:09
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 MB 1.9 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.7/1

In [7]:
import cudf
cudf.__version__

'23.12.01'

In [1]:
import cuml
cuml.__version__

/home/g4/ML shit/venv/lib/python3.10/site-packages/cupy/_environment.py:487: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda11x, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


'24.02.00'

In [ ]:
import cudf
import numpy as np
from cuml.ensemble import RandomForestRegressor
from cuml.common.device_selection import using_device_type
from cuml.common.device_selection import set_global_device_type, get_global_device_type

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import cupy as cp
set_global_device_type('gpu')

# Read the dataset
file_path = '/content/drive/Shareddrives/Machine_Learning Data cleaning/north_Final.csv'
df = pd.read_csv(file_path)

# Define features and target variable
X = df.drop(columns=['Air temperature (instant) in celsius degrees'])
y = df['Air temperature (instant) in celsius degrees']

# Convert data to cuDF format
X_cudf = cudf.DataFrame.from_pandas(X)
y_cudf = cudf.Series(y)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_cudf, y_cudf, test_size=0.2, random_state=42)

# Train the random forest model
model = RandomForestRegressor(n_estimators=100)
# with using_device_type('gpu'):
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Convert predictions to pandas DataFrame
y_pred_pd = y_pred.to_pandas()

# Evaluate the model
mse = mean_squared_error(cp.asnumpy(y_test), y_pred_pd.values)
r2 = r2_score(cp.asnumpy(y_test), y_pred_pd.values)

print("Mean Squared Error (MSE) for Random Forest (GPU):", mse)
print("R-squared (R2) for Random Forest (GPU):", r2)


/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


Mean Squared Error (MSE) for Random Forest (GPU): 0.1077971791738604
R-squared (R2) for Random Forest (GPU): 0.9915964931198051


## with n_estimators = 200

In [9]:
import cudf
import numpy as np
from cuml.ensemble import RandomForestRegressor
from cuml.common.device_selection import using_device_type
from cuml.common.device_selection import set_global_device_type, get_global_device_type

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import cupy as cp
set_global_device_type('gpu')

# Read the dataset
file_path = 'north_Final.csv'
df = pd.read_csv(file_path)

# Define features and target variable
X = df.drop(columns=['Air temperature (instant) in celsius degrees'])
y = df['Air temperature (instant) in celsius degrees']

# Convert data to cuDF format
X_cudf = cudf.DataFrame.from_pandas(X)
y_cudf = cudf.Series(y)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_cudf, y_cudf, test_size=0.2, random_state=42)

# Train the random forest model
model = RandomForestRegressor(n_estimators=200)
# with using_device_type('gpu'):
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Convert predictions to pandas DataFrame
y_pred_pd = y_pred.to_pandas()

# Evaluate the model
mse = mean_squared_error(cp.asnumpy(y_test), y_pred_pd.values)
r2 = r2_score(cp.asnumpy(y_test), y_pred_pd.values)

print("Mean Squared Error (MSE) for Random Forest (GPU):", mse)
print("R-squared (R2) for Random Forest (GPU):", r2)


/home/g4/ML shit/venv/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


Mean Squared Error (MSE) for Random Forest (GPU): 0.10782241079127787
R-squared (R2) for Random Forest (GPU): 0.9915945261474577


# **5. Neural Network Regression model with 30 epochs , 64 dense layers and non linear activation**

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

# Read the dataset
file_path = 'north_Final.csv'
df = pd.read_csv(file_path)

# Define features and target variable
X = df.drop(columns=['Air temperature (instant) in celsius degrees'])  # Features
y = df['Air temperature (instant) in celsius degrees']  # Target variable

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Build the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=2)

# Make predictions
y_pred = model.predict(X_test).flatten()

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE) for Perceptron Neural Network:", mse)
print("R-squared (R2) for Perceptron Neural Network:", r2)


2024-04-07 22:53:50.644396: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 22:53:51.433669: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-07 22:54:15.520121: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-07 22:54:15.520628: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. 

Epoch 1/30
209808/209808 - 116s - 553us/step - loss: 0.8098
Epoch 2/30
209808/209808 - 110s - 522us/step - loss: 0.3340
Epoch 3/30
209808/209808 - 112s - 536us/step - loss: 0.3067
Epoch 4/30
209808/209808 - 106s - 504us/step - loss: 0.2894
Epoch 5/30
209808/209808 - 115s - 550us/step - loss: 0.2792
Epoch 6/30
209808/209808 - 127s - 606us/step - loss: 0.2731
Epoch 7/30
209808/209808 - 106s - 505us/step - loss: 0.2683
Epoch 8/30
209808/209808 - 124s - 593us/step - loss: 0.2631
Epoch 9/30
209808/209808 - 112s - 534us/step - loss: 0.2588
Epoch 10/30
209808/209808 - 113s - 537us/step - loss: 0.2546
Epoch 11/30
209808/209808 - 115s - 546us/step - loss: 0.2519
Epoch 12/30
209808/209808 - 113s - 540us/step - loss: 0.2476
Epoch 13/30
209808/209808 - 113s - 538us/step - loss: 0.2448
Epoch 14/30
209808/209808 - 105s - 500us/step - loss: 0.2415
Epoch 15/30
209808/209808 - 111s - 530us/step - loss: 0.2395
Epoch 16/30
209808/209808 - 115s - 550us/step - loss: 0.2377
Epoch 17/30
209808/209808 - 110s 

# **Classification**
## ***Binning the label in order to turn it into a classifcation problem***

In [5]:
import pandas as pd

# Load your dataset
# file_path = 'north_Final_sampled.csv'
file_path = 'north_Final.csv'

df = pd.read_csv(file_path)

# Define bins and labels
bins = [-10, 0, 10, 20, 30, 40, 50]
labels = ['Very Cold', 'Cold', 'Moderate', 'Warm', 'Hot', 'Very Hot']

# Bin the target variable
df['Temperature_Category'] = pd.cut(df['Air temperature (instant) in celsius degrees'], bins=bins, labels=labels)
df.drop(columns=['Air temperature (instant) in celsius degrees'], inplace=True)

# Check the distribution of categories
print(df['Temperature_Category'].value_counts())

df.to_csv('north_Final_classfied.csv')


Warm         6683929
Hot          1550142
Moderate      155500
Cold            2561
Very Hot         152
Very Cold         15
Name: Temperature_Category, dtype: int64


## **Neural Network Model With a Softmax Layer**

In [7]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.python.client import device_lib

# Get list of all available devices
local_device_protos = device_lib.list_local_devices()

# Check if GPU is available
gpu_available = any(device.device_type == 'GPU' for device in local_device_protos)

if gpu_available:
    print("GPU available. Training on GPU...")
else:
    print("No GPU available. Training on CPU...")
# Read the dataset
file_path = 'north_Final_classfied.csv'
df = pd.read_csv(file_path)

# Define features and target variable
X = df.drop(columns=['Temperature_Category'])
y = df['Temperature_Category']

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Print the mapping of labels to numbers
print("Label Encoding Mapping:")
for label, number in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
    print(f"{label}: {number}")

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Build the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)

# Convert encoded labels back to original labels
y_test_original = label_encoder.inverse_transform(y_test)
y_pred_original = label_encoder.inverse_transform(y_pred_classes)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test_original, y_pred_original, average='weighted')
recall = recall_score(y_test_original, y_pred_original, average='weighted')
f1 = f1_score(y_test_original, y_pred_original, average='weighted')

# Calculate accuracy
accuracy = accuracy_score(y_test_original, y_pred_original)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Accuracy:", accuracy)


2024-04-07 22:04:36.950448: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 22:04:37.977360: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-07 22:04:38.638767: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-07 22:04:38.639314: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. 

No GPU available. Training on CPU...
Label Encoding Mapping:
Cold: 0
Hot: 1
Moderate: 2
Very Cold: 3
Very Hot: 4
Warm: 5


2024-04-07 22:05:04.631257: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-07 22:05:04.632375: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-07 22:05:04.633135: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required l

Epoch 1/10


2024-04-07 22:05:05.279083: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 778805324 exceeds 10% of free system memory.


209808/209808 - 116s - 551us/step - accuracy: 0.7612 - loss: 483.5031
Epoch 2/10
209808/209808 - 111s - 529us/step - accuracy: 0.7965 - loss: 0.7006
Epoch 3/10
209808/209808 - 116s - 554us/step - accuracy: 0.7966 - loss: 0.5986
Epoch 4/10
209808/209808 - 121s - 575us/step - accuracy: 0.7966 - loss: 0.6166
Epoch 5/10
209808/209808 - 115s - 549us/step - accuracy: 0.7965 - loss: 0.6177
Epoch 6/10
209808/209808 - 119s - 568us/step - accuracy: 0.7965 - loss: 0.6070
Epoch 7/10
209808/209808 - 121s - 579us/step - accuracy: 0.7966 - loss: 0.5969
Epoch 8/10
209808/209808 - 121s - 577us/step - accuracy: 0.7966 - loss: 0.5902
Epoch 9/10
209808/209808 - 122s - 579us/step - accuracy: 0.7966 - loss: 0.5948
Epoch 10/10
209808/209808 - 119s - 567us/step - accuracy: 0.7966 - loss: 0.6047
Test Loss: 0.5704593062400818
Test Accuracy: 0.796042799949646
52452/52452 ━━━━━━━━━━━━━━━━━━━━ 28s 533us/step


/home/g4/ML shit/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Precision: 0.7707001955834194
Recall: 0.7960428011391395
F1-score: 0.7057212545787996
Accuracy: 0.7960428011391395
